In [12]:
import numpy as np
import pandas as pd
import math

#### Lasso regression
* parameter 선정 : 에러율(MSE) 적을 때로 선택(0.0005)

![lasso](./lasso.png)

In [ ]:
# lasso reg for sentiment dictionary

for x in range(len(score)):
    if math.isnan(score[x]) == True:
        score[x] = 0

final_lasso = linear_model.Lasso(alpha=0.0005)
final_lasso.fit(words, score)

fea_score = [[feature, coef] for feature, coef in
             zip(list(countvectorizer.get_feature_names()), list(final_lasso.coef_))]
fea_score = pd.DataFrame(np.array(fea_score))
fea_score.columns = ['feature', 'sen_score']
fea_score['sen_score'] = pd.to_numeric(fea_score['sen_score'])
fea_score = fea_score[(fea_score['sen_score'] > 0) | (fea_score['sen_score'] < 0)]

In [13]:
# 감정 사전 개수 : 2568
fea_score = pd.read_csv(r"C:\Users\User\PycharmProjects\word2vec\new_fea_score.csv",encoding='cp949')
fea_score.sort_values(by=['sen_score'],axis=0,ascending=False)[:10]

,feature,sen_score
157,awsom,1.140084
2308,trevor,1.026796
2459,warthog,0.971381
32,addict,0.874063
2232,th,0.869647
2485,whiner,0.865695
456,coolest,0.863885
2015,skeptic,0.860849
778,fabul,0.853166
1817,refil,0.851272


In [14]:
fea_score.sort_values(by=['sen_score'],axis=0)[:10]

,feature,sen_score
497,crimefest,-2.328739
896,fraud,-1.892681
2102,stalin,-1.672318
2384,unplay,-1.632498
1993,sicken,-1.517665
1295,lever,-1.486803
1821,refund,-1.472257
2120,stinker,-1.469982
609,disgrac,-1.466308
1881,rigid,-1.402668


#### 속성별 감정 점수 산출 방법
* 문서별 감정 단어가 나타나는 위치를 파악
* 나타난 위치에서 N-gram 앞뒤 단어들을 파악
* 그 중 몇 가지 우선순위 조건하에 한 감정 단어당 하나의 주제에 해당 가중치가 부여
    1. 감정 단어의 앞뒤로 토픽 관련 단어가 존재하는 경우 앞의 단어를 우선한다. 
    2. 감정 단어와 가까운 위치에 존재하는 단어를 우선한다. 

![senti_process](./senti_process.png)

* ‘맛‘이라는 단어의 감정이 5점일 때, ‘음식의 질‘(p=0.8) 과 ‘모양‘(p=0.2) 중 단어 ‘맛’에 대한 감정 값이 음식의 질에 대한 평가였을 확률이 크므로 전자에 5 * 0.8=4점, 후자는 1점을 부여

In [ ]:
# Process - 감정단어를 검색하고 앞뒤 n개 단어를 searching

n = 3
dws = []
for d, docu in enumerate(snowball3_senti):
    for plo in sentiment_list:
        plo_score = list(fea_score[fea_score['feature'] == plo]['sen_score'])[0]
        plo_idx = [i for i, w in enumerate(docu) if w == plo]
        for idx in plo_idx:
            s_idx = np.where(idx - n < 0, 0, idx - n)
            e_idx = np.where(idx + n + 1 > len(docu), len(docu), idx + n + 1)
            f_ngram = docu[s_idx:idx]
            b_ngram = docu[idx + 1:e_idx]

            if len(f_ngram) != 0:
                topic_idx = [i for i, w in enumerate(f_ngram) if w in vocab]
                if len(topic_idx) != 0:
                    topic_words = f_ngram[np.max(topic_idx)]
                    twi = vocab.index(topic_words)
                    dws.append([d, twi, plo_score])
            elif len(b_ngram) != 0:
                topic_idx = [i for i, w in enumerate(b_ngram) if w in vocab]
                if len(topic_idx) != 0:
                    topic_words = b_ngram[np.min(topic_idx)]
                    twi = vocab.index(topic_words)
                    dws.append([d, twi, plo_score])
            else:
                next
    print(d)

dwsm = np.zeros(shape=(d + 1, len(vocab)))
for i in range(0, len(dws)):
    dwsm[dws[i][0]][dws[i][1]] = dwsm[dws[i][0]][dws[i][1]] + dws[i][2]


In [ ]:
senti_table = pd.read_csv(r"C:\Users\User\Desktop\game_word2vec\dwsm.csv",encoding='cp949')

#게임/토픽별 점수
grouped=X.groupby(['kind_of_games'])
game_num=[0]+list(grouped.last().sort_values('id')['id']+1)
game_num = list(X.index)
game_topic=[]
for i in range(0,len(game_num)-1):
    if game_num[i+1] <= d+1:
        s_num=game_num[i]
        f_num=game_num[i+1]
        game_topic.append(list(np.dot(senti_table[s_num:f_num],ratio_topic.T).sum(axis=0)))
    else:
        next

In [17]:
lda_topic = pd.read_csv(r"C:\Users\User\Desktop\game_word2vec\lda_topic.csv",encoding = 'cp949')
lda_topic

,Unnamed: 0,Topic_N,Words,Unnamed: 3
0,0,0,['play' 'graphics' 'thing' 'player' 'first' 'f...,그래픽
1,1,1,['story' 'pc' 'multiplayer' 'system' 'players'...,스토리
2,2,2,['characters' 'old' 'real' 'only' 'world' 'tea...,캐릭터
3,3,3,['weapons' 'fps' 'few' 'best' 'fantastic' 'spa...,1인칭 게임
4,4,4,['little' 'story' 'maps' 'original' 'weapon' '...,레벨 디자인
5,5,5,['level' 'main' 'own' 'big' 'excellent' 'score...,레벨 디자인2
6,6,6,['fun' 'bugs' 'content' 'mode' 'enjoyable' 'th...,"완성도(버그, 콘텐츠 등)"
7,7,7,['graphics' 'something' 'engine' 'voice' 'titl...,그래픽
8,8,8,['short' 'play' 'series' 'character' 'end' 'pe...,시리즈
9,9,9,['ai' 'story' 'people' 'few' 'money' 'differen...,인공지능(캐릭터)


In [11]:
path="C:/Users/User/Desktop/game_word2vec/"
topic = pd.read_csv(path+'game_topic_sen.csv',names=['graphic','story','character','first_person','level_design','levle_design2','perfection','graphic2','series','AI','play','play2'])
topic.head(10)

,graphic,story,character,first_person,level_design,levle_design2,perfection,graphic2,series,AI,play,play2
0,0.003719,0.001212,0.000803,4.089695e-04,0.001860,0.002165,0.001478,0.004158,0.003478,0.001348,0.000688,0.005337
1,0.000613,0.002169,0.002580,7.295300e-04,0.002665,0.000635,0.001164,0.018869,0.009543,0.002191,0.000855,0.010085
2,0.000415,0.000173,-0.000546,-2.519551e-05,0.000861,0.002647,0.003095,0.001832,0.001226,-0.000840,0.003454,-0.003925
3,0.002379,0.002195,0.005555,-1.111952e-04,0.001781,0.002320,0.001895,0.006379,0.003089,0.002420,0.001356,0.014480
4,-0.000027,-0.000110,-0.000333,-4.631036e-08,-0.000450,-0.000172,-0.000611,-0.000206,-0.000311,-0.000205,-0.000007,-0.000335
5,0.003701,0.005807,0.007760,5.474243e-04,0.003783,0.004074,0.005294,0.007182,0.008626,0.004477,0.001561,0.029148
6,0.001032,0.001705,0.002088,8.922178e-05,0.001713,0.001547,0.000797,0.002097,0.002015,0.001043,0.000865,0.006500
7,-0.000039,0.001862,0.004595,-2.581344e-04,0.001156,0.002969,0.000635,0.002152,0.005462,0.000648,0.000161,0.022605
8,0.008134,0.000390,0.001093,1.544897e-06,0.001896,0.001741,0.003123,0.003162,0.001483,0.000338,0.000747,0.005732
9,-0.000396,-0.000296,-0.000023,-2.034846e-07,0.000139,-0.000181,0.000800,-0.000524,-0.000679,0.000386,0.000086,-0.001200


In [ ]:
game_seq = X['kind_of_games']
game_table = pd.concat([game_seq,topic],axis=1)

senti_game_score = game_table[game_table.columns[1:]].groupby(game_table['kind_of_games']).sum()

In [16]:
senti_game_score = pd.read_csv(r"C:\Users\User\PycharmProjects\word2vec\topic_senti_score.csv",encoding='cp949')
senti_game_score.head(10)

,kind_of_games,graphic,story,character,first_person,level_design,levle_design2,perfection,graphic2,series,AI,play,play2
0,007 Legends,-0.171563,-0.072082,-0.163929,-0.014126,-0.104099,-0.089794,-0.161013,-0.146605,-0.087366,-0.114761,-0.099013,-0.163298
1,007: NightFire,-0.031806,0.028107,0.003372,-0.000953,-0.004948,-0.015296,-0.003538,0.000071,-0.022329,-0.004255,-0.019583,-0.024999
2,007: Quantum of Solace,-0.004620,-0.002360,-0.025032,0.000541,0.002555,-0.003534,-0.014814,-0.009813,-0.005692,-0.008656,0.000021,-0.036461
3,10 Second Ninja,-0.000298,-0.000662,-0.001971,0.000786,0.000555,-0.001185,-0.000734,-0.001006,-0.000837,-0.001448,-0.001095,-0.001330
4,1001 Spikes,0.000929,0.004003,0.006719,0.000436,-0.000992,0.000889,0.002416,0.004963,0.005116,0.002586,0.000984,0.002932
5,12 is Better Than 6,0.000291,0.010753,0.025709,-0.000419,0.017734,0.012103,0.011098,0.020797,0.020325,0.008032,0.014471,0.009895
6,140,0.029263,0.036557,0.107933,0.001571,0.077840,0.025263,0.047170,0.049288,0.047550,0.049004,0.038419,0.039362
7,25 to Life,-0.014079,-0.003123,-0.023045,-0.000816,-0.015060,-0.003393,-0.005819,-0.006767,-0.007310,-0.021919,-0.006966,-0.028371
8,7554,-0.001005,0.001843,-0.000941,0.000081,0.002341,0.001194,-0.000746,0.000215,0.001170,-0.005727,0.002527,0.000508
9,8BitBoy,0.006655,0.000793,0.000544,0.000633,0.003454,0.008251,0.007598,0.004290,0.005036,0.001613,0.011786,0.006929
